# NER4ID at SemEval-2022 Task 2: Named Entity Recognition for Idiomaticity Detection
----------------------------------------------------------------------------------------------------

In [ ]:
!git clone https://github.com/babelscape/ner4id

Cloning into 'ner4id'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 56 (delta 20), reused 41 (delta 10), pack-reused 0
Unpacking objects: 100% (56/56), 1.58 MiB | 2.99 MiB/s, done.


## Dependencies

In [ ]:
! pip install transformers
! pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


In [ ]:
import os

from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
 
from pprint import pprint
import random
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertModel, BertConfig
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

In [ ]:
os.chdir("../")

In [ ]:
print(torch.__version__)
torch.cuda.current_device()
torch.cuda.get_device_name(0)

2.0.0+cu118


'Tesla T4'

## Read Data

In [ ]:
mode = input("Do you want to train a model from scratch or do you want to evaluate a pretrained model? Possible answers: ['train', 'eval']\n> ")
assert mode in ['train', 'eval']

setting = input("\nIn which setting do you want to train/evaluate your model? Possible answers: ['zero-shot', 'one-shot']\n>")
assert setting in ['zero-shot', 'one-shot']

Do you want to train a model from scratch or do you want to evaluate a pretrained model? Possible answers: ['train', 'eval']
> train

In which setting do you want to train/evaluate your model? Possible answers: ['zero-shot', 'one-shot']
>zero-shot


In [ ]:
train_file_zero_shot = "/content/ner4id/data/train_zero_shot.csv" 
train_file_one_shot = "/content/ner4id/data/train_one_shot.csv"

dev_file = "/content/ner4id/data/dev.csv"
dev_file_gold = "/content/ner4id/data/dev_gold.csv"
test_file = "/content/ner4id/data/test.csv"

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy download pt_core_news_sm

2023-05-09 23:34:16.157183: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 37.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import spacy
import en_core_web_sm
import pt_core_news_sm
from spacy.cli.download import download as spacy_download

spacy_download("en_core_web_sm")
spacy_tagger_en = spacy.load("en_core_web_sm", exclude=["ner", "parser"])

spacy_download("pt_core_news_sm")
spacy_tagger_pt = spacy.load("pt_core_news_sm", exclude=["ner", "parser"])

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
import csv

train_data = {}
idx = 0

predictions = []
labels = []

if setting=="zero-shot":
    with open(train_file_zero_shot, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in tqdm(reader):
            
            if len((row[4].strip() + row[5].strip() + row[6].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()+ " " + row[6].strip()
            elif len((row[4].strip() + row[5].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()
            else:
                text = row[5].strip()
                
            e = row[2].strip()

            train_data[idx] = {
                                "id": row[0],
                                "lang": row[1],
                                "expression": e,
                                "text": row[5].strip(),
                                "idiomatic": True if row[7] == "0" else False
                            }    
            idx += 1


if setting=="one-shot":
    with open(train_file_one_shot, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in tqdm(reader):
            
            if len((row[4].strip() + row[5].strip() + row[6].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()+ " " + row[6].strip()
            elif len((row[4].strip() + row[5].strip()).split(" ")) < 300:
                text = row[4].strip()+ " " + row[5].strip()
            else:
                text = row[5].strip()
                
            e = row[2].strip()
                        
            train_data[idx] = {
                                "id": row[0],
                                "lang": row[1],
                                "expression": e,
                                "text": row[5].strip(),
                                "idiomatic": True if row[7] == "0" else False
                            }    
            idx += 1

print(len(train_data)) 

4492it [00:00, 64919.52it/s]

4492


In [ ]:
labels_dev = {}

with open(dev_file_gold, newline='',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        labels_dev[row[0]] = True if row[3] == "0" else False

dev_data = {}
idx = 0

with open(dev_file, newline='',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:

        if len((row[3].strip() + row[4].strip() + row[5].strip()).split(" ")) < 300:
            text = row[3].strip() + " " + row[4].strip()+ " " + row[5].strip()
        elif len((row[3].strip() + row[4].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()
        else:
            text = row[4].strip()
        
        e = row[2].strip()
                
        
        dev_data[idx] = {
                        "id": row[0],
                        "lang": row[1],
                        "expression": e,
                        "text": row[4].strip(),
                        "idiomatic": labels_dev[row[0]]
                        }
                
        idx += 1
        
    
print(len(dev_data)) 

740


In [ ]:
test_data = {}
idx = 0

with open(test_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:

        if len((row[3].strip() + row[4].strip() + row[5].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()+ " " + row[5].strip()
        elif len((row[3].strip() + row[4].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()
        else:
            text = row[4].strip()
        
        e = row[2].strip()

        test_data[idx] = {
                            "id": row[0],
                            "lang": row[1],
                            "expression": e,
                            "text": row[4].strip(),
                            "idiomatic": True #it's just a FAKE label used to maintain the same structure of the dataset entries, 
                                              #it could be also False
                        }
                
        idx += 1
        
    
print(len(test_data)) 

2343


## Fix Random Seed

In [ ]:
SEED = 2 #we set a seed for having replicability of results
 
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Load BERT Model

In [ ]:
model_name = 'bert-base-multilingual-cased'
 
bert_config = BertConfig.from_pretrained(model_name, output_hidden_states=True)
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name, config=bert_config)

In [ ]:
from transformers import AutoTokenizer, TFXLMRobertaForCausalLM
import tensorflow as tf

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = TFXLMRobertaForCausalLM.from_pretrained("xlm-roberta-base")


If you want to use `TFXLMRobertaLMHeadModel` as a standalone, add `is_decoder=True.`
All model checkpoint layers were used when initializing TFXLMRobertaForCausalLM.

Some layers of TFXLMRobertaForCausalLM were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['lm_head']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Dataset

In [ ]:
nlp_en = en_core_web_sm.load()
nlp_pt = pt_core_news_sm.load()

class IdiomDataset(Dataset):
    def __init__(self, 
                 dataset, 
                 tokenizer,
                 languages,
                 device="cuda",
                ) -> None:
        
        self.encoded_data = []
    
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.languages = languages
        self.device = device
        self.__init_encoded_data()

 
    def __init_encoded_data(self):
                
        for idx in tqdm(self.dataset):
            id = self.dataset[idx]["id"]
            lang = self.dataset[idx]["lang"]
            e = self.dataset[idx]["expression"]
            label = self.dataset[idx]["idiomatic"]
            context_tmp = self.dataset[idx]["text"]

            if lang == "EN":
                doc = nlp_en(context_tmp)
                ents = [X.text.lower() for X in doc.ents]
            else:
                doc = nlp_pt(context_tmp)
                ents = [X.text.lower() for X in doc.ents]
            
    
            if e in ents:
                e = context_tmp
                context = context_tmp
            else:
                if context_tmp.find(e) != -1:
                    context = context_tmp[:context_tmp.lower().find(e)] + context_tmp[context_tmp.lower().find(e)+len(e):]

                else:    
                    e = context_tmp
                    context = context_tmp                     
                
        
            tokenized_e = torch.tensor(self.tokenize_mention(e, self.tokenizer, True))
            tokenized_context = torch.tensor(self.tokenize_mention(context, self.tokenizer, True))

            if e!="MWE" and lang in self.languages:
                self.encoded_data.append((idx,
                                          e,
                                          context,
                                          tokenized_e,
                                          tokenized_context,
                                          torch.tensor([-1.0]) if label == True else torch.tensor([1.0]),
                                          id, 
                                          lang))

     
    def tokenize_mention(self, sent, tokenizer, special_tokens):
        encoded_sentence = tokenizer.encode(sent, add_special_tokens = special_tokens)
        return encoded_sentence[:500]
    
    def tokenize_description(self, sent, tokenizer, window):
        encoded_sentence = tokenizer.encode(sent, add_special_tokens = True)
        return encoded_sentence

    def __len__(self):
        return len(self.encoded_data)
 
    def __getitem__(self, idx: int):
        return self.encoded_data[idx]

Create the dataset:

In [ ]:
train_dataset = IdiomDataset(train_data, tokenizer, languages = ["EN", "PT", "GL"])
dev_dataset = IdiomDataset(dev_data, tokenizer, languages =["EN", "PT", "GL"])
test_dataset = IdiomDataset(test_data, tokenizer, languages = ["EN","PT", "GL"])

print(len(train_dataset))
print(len(dev_dataset))
print(len(test_dataset))

100%|██████████| 2343/2343 [00:25<00:00, 92.72it/s] 

4491
739
2342


Create the dataloader:

In [ ]:
def collate(elems: tuple) -> tuple:
    ids, e, texts, expressions, contexts, labels, ids, langs = list(zip(*elems))
    
    pad_expressions = pad_sequence(expressions, batch_first=True, padding_value=0)
    pad_contexts = pad_sequence(contexts, batch_first=True, padding_value=0)
    pad_labels = pad_sequence(labels, batch_first=True, padding_value=0)
 
    return ids, e, texts, pad_expressions, pad_contexts, pad_labels.cuda(), ids, langs


train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate)
dev_dataloader = DataLoader(dev_dataset, batch_size=8, shuffle=False, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate)

print(len(train_dataloader))
print(len(dev_dataloader))
print(len(test_dataloader))

562
93
293


## Dual-Encoder Architecture

In [ ]:
from transformers import XLMRobertaModel
model_name = "xlm-roberta-base"

In [ ]:
class DualEncoder(nn.Module):
    def __init__(self, hparams):
        super(DualEncoder, self).__init__()
        pprint(params)
 
        self.hparams = hparams
 
        
        self.expression_encoder = XLMRobertaModel.from_pretrained(model_name)
        self.context_encoder = XLMRobertaModel.from_pretrained(model_name)

        self.cosine_similarity = nn.CosineSimilarity(dim=-1, eps=1e-6)
        
        self.dropout = nn.Dropout(hparams.dropout)

        self.tanh = nn.Tanh()
            
        for param in self.context_encoder.parameters():
            param.requires_grad = False
  

    def forward(self, expression, context, mask1, mask2):
        
        embedding_context = self.context_encoder.forward(context.cuda(), mask2.cuda())[0]
        embedding_context = embedding_context[:,0,:].squeeze(1)

        embedding_expression = self.expression_encoder.forward(expression.cuda(), mask1.cuda())[0]
        embedding_expression = torch.sum(embedding_expression, 1)
            
        similarities = self.cosine_similarity(embedding_expression, embedding_context) 
                        
        return similarities

In [ ]:
class HParams():
    dropout = 0.25
    
params = HParams()

Instantiate the model: 

In [ ]:
my_model = DualEncoder(params).cuda()
my_model

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are ini

DualEncoder(
  (expression_encoder): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
 

## Trainer

In [ ]:
class Trainer():
    def __init__(self,
                model:nn.Module, 
                loss_function,
                optimizer,
                gradient_accumulation_steps):
        
        self.model = model
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.gradient_accumulation_steps = gradient_accumulation_steps
 
    def padding_mask(self, batch):
        padding = torch.ones_like(batch)
        padding[batch == 0] = 0
        padding = padding.type(torch.uint8)
        return padding
 
    def train(self,
            train_dataset:Dataset, 
            valid_dataset:Dataset,
            epochs:int=1,
            patience:int=10,
            modelname=setting):
        
        print("\nTraining...")
 
        record_dev = 0.0
        full_patience = patience
        modelname = modelname
 
        for epoch in range(epochs):
             if patience>0:
                print(" Epoch {:03d}".format(epoch + 1))

                epoch_loss = 0.0
                self.model.train()
                
                count_batches = 0
                self.optimizer.zero_grad()
                
                for ids, e, text, expressions, contexts, labels, id, lang in tqdm(train_dataset):
                    batch_loss = 0.0

                    mask1 = self.padding_mask(expressions)
                    mask2 = self.padding_mask(contexts)
                    
                    similarities = self.model(expressions, contexts, mask1, mask2)

                    labels = labels.view(-1)
                    batch_loss = self.loss_function(similarities, labels)
                    epoch_loss += batch_loss

                    batch_loss.backward()

                    if count_batches % self.gradient_accumulation_steps == 0:
                        self.optimizer.step()
                        self.optimizer.zero_grad()


                avg_epoch_loss = epoch_loss / len(train_dataset)
                print('[E: {:2d}] train loss = {:0.4f}'.format(epoch+1, avg_epoch_loss))

                valid_loss, f1 = self.evaluate(valid_dataset, epoch)

                if f1>record_dev:
                    record_dev = f1
                    torch.save(self.model.state_dict(), "/content/ner4id/checkpoints/"+modelname+".pt")
                    patience = full_patience
                else:
                    patience -= 1
                   
                print('\t[E: {:2d}] valid loss = {:0.4f}, f1-score = {:0.4f}, patience: {:2d}'.format(epoch+1, valid_loss, f1, patience))


        print("...Done!")

        return avg_epoch_loss

    def evaluate(self, valid_dataset, epoch):

        valid_loss = 0.0
        all_predictions = list()
        all_labels = list()
        
        predictions = {}
         
        self.model.eval()
                    
        for ids, e, text, expressions, contexts, labels, id, lang in tqdm(valid_dataset):
            mask1 = self.padding_mask(expressions)
            mask2 = self.padding_mask(contexts)
            
            with torch.no_grad():
                similarities = self.model(expressions, contexts, mask1, mask2)

            labels = labels.view(-1)
            loss = self.loss_function(similarities, labels)
 
            for i in range(len(similarities)):
                if similarities[i]>0:
                    all_predictions.append(1)
                else:
                    all_predictions.append(0)

                all_labels.append(1 if labels[i].item()==1 else 0)
            
            valid_loss += loss
            
            for i in range(len(similarities)):
                if similarities[i]>0:
                    predictions[ids[i]] = 1

                else:
                    predictions[ids[i]] = 0

        f1 = f1_score(all_labels, all_predictions, average= 'macro')
        print(classification_report(all_labels, all_predictions, digits=4))
        
        return valid_loss / len(valid_dataset), f1

Instantiate the trainer:

In [ ]:
trainer = Trainer(model = my_model,
                    loss_function = nn.MSELoss(),
                    optimizer = optim.Adam(my_model.parameters(), lr=0.00001),
                    gradient_accumulation_steps=1)

Train the system:

In [ ]:
if mode == "train":
    trainer.train(train_dataloader, dev_dataloader, epochs = 1000, patience = 5)


Training...
 Epoch 001


100%|██████████| 562/562 [01:40<00:00,  5.59it/s]


[E:  1] train loss = 0.1038


100%|██████████| 93/93 [00:04<00:00, 19.16it/s]


              precision    recall  f1-score   support

           0     0.6556    0.8839    0.7529       336
           1     0.8636    0.6129    0.7170       403

    accuracy                         0.7361       739
   macro avg     0.7596    0.7484    0.7349       739
weighted avg     0.7691    0.7361    0.7333       739

	[E:  1] valid loss = 1.0024, f1-score = 0.7349, patience:  5
 Epoch 002


...

              precision    recall  f1-score   support

           0     0.6631    0.9137    0.7685       336
           1     0.8949    0.6129    0.7275       403

    accuracy                         0.7497       739
   macro avg     0.7790    0.7633    0.7480       739
weighted avg     0.7895    0.7497    0.7461       739

	[E: 14] valid loss = 0.9723, f1-score = 0.7480, patience:  1
 Epoch 015


100%|██████████| 562/562 [01:43<00:00,  5.44it/s]


[E: 15] train loss = 0.1046


100%|██████████| 93/93 [00:04<00:00, 19.64it/s]


              precision    recall  f1-score   support

           0     0.6395    0.8869    0.7431       336
           1     0.8608    0.5831    0.6953       403

    accuracy                         0.7212       739
   macro avg     0.7501    0.7350    0.7192       739
weighted avg     0.7602    0.7212    0.7170       739

	[E: 15] valid loss = 0.9230, f1-score = 0.7192, patience:  0
...Done!


## Predict the test set

Load the best model checkpoint:

In [ ]:
my_model.load_state_dict(torch.load(f"/content/ner4id/checkpoints/{setting}.pt"))

and then use the model to predict the test set:

In [ ]:
import os
folder_name = "predictions"
par_dir = "/content/ner4id"
path = os.path.join(par_dir, folder_name)
os.makedirs(path)

In [ ]:
def padding_mask(batch):
        padding = torch.ones_like(batch)
        padding[batch == 0] = 0
        padding = padding.type(torch.uint8)
        return padding

def predict(model, test_dataset):

    predictions = {}
        
    model.eval()
                
    for ids, e, text, expressions, contexts, labels, id, lang in tqdm(test_dataset):
        mask1 = padding_mask(expressions)
        mask2 = padding_mask(contexts)
        
        with torch.no_grad():
            similarities = model(expressions, contexts, mask1, mask2)

        labels = labels.view(-1)

        for i in range(len(similarities)):
            if similarities[i]>0:
                predictions[ids[i]] = 1

            else:
                predictions[ids[i]] = 0


    predictions["ID"] = "Label"

    if setting == "zero-shot":
        with open("/content/ner4id/data/test_submission_format.csv") as csvfile:
            with open("/content/ner4id/predictions/task2_subtaska.csv", "w") as out:
                reader = csv.reader(csvfile, delimiter=',')
                for row in reader:
                    if row[2] == "zero_shot" or row[0] == "ID":
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + str(predictions[row[0]]) + "\n")
                    else:
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + "" + "\n")
    else:
        with open("/content/ner4id/data/test_submission_format.csv") as csvfile:
            with open("/content/ner4id/predictions/task2_subtaska.csv", "w") as out:
                reader = csv.reader(csvfile, delimiter=',')
                for row in reader:
                    if row[2] == "one_shot" or row[0] == "ID":
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + str(predictions[row[0]]) + "\n")
                    else:
                        out.write(row[0] + "," + row[1] + "," + row[2] + "," + "" + "\n")    


predict(my_model, test_dataloader)

100%|██████████| 293/293 [00:13<00:00, 20.93it/s]


The output file (saved in the /predictions folder) contains predictions in the standard format specified by the competition rules. You can now upload the output file on CodaLab and see the scores obtained by the system on the test set.

## Multilingual testing




In [ ]:
multi_dataset = "/content/ner4id/data/final_multi_lingual_spanish.csv"
multi_data = {}
idx = 0

with open(multi_dataset, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:

        if len((row[3].strip() + row[4].strip() + row[5].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()+ " " + row[5].strip()
        elif len((row[3].strip() + row[4].strip()).split(" ")) < 300:
            text = row[3].strip()+ " " + row[4].strip()
        else:
            text = row[4].strip()
        
        e = row[2].strip()

        multi_data[idx] = {
                            "id": row[0],
                            "lang": row[1],
                            "expression": e,
                            "text": row[4].strip(),
                            "idiomatic": True #it's just a FAKE label used to maintain the same structure of the dataset entries, 
                                              #it could be also False
                        }
                
        idx += 1
        
    
print(len(multi_data)) 


21


In [ ]:
multi_dataset = IdiomDataset(multi_data, tokenizer, languages = ["EN","PT", "GL", "ESP"])
robust_dataloader = DataLoader(multi_dataset, shuffle=False, collate_fn=collate)
print(len(robust_dataloader))

100%|██████████| 21/21 [00:00<00:00, 36.87it/s]

20


In [ ]:
def predict_robust(model, test_dataset):

    predictions = {}
        
    model.eval()
                
    for ids, e, text, expressions, contexts, labels, id, lang in tqdm(test_dataset):
        mask1 = padding_mask(expressions)
        mask2 = padding_mask(contexts)
        
        with torch.no_grad():
            similarities = model(expressions, contexts, mask1, mask2)

        labels = labels.view(-1)

        for i in range(len(similarities)):
            if similarities[i]>0:
                predictions[ids[i]] = 1

            else:
                predictions[ids[i]] = 0
    return predictions

In [ ]:
predictions = predict_robust(my_model, robust_dataloader)

100%|██████████| 20/20 [00:00<00:00, 41.32it/s]


In [ ]:
import pandas as pd
actual = pd.read_csv("/content/ner4id/data/spanish_key - Sheet1.csv")
actual_values = actual['Label'].tolist()
len(actual_values)

20

In [ ]:
list_pred = predictions.values()
list_pred = list(list_pred)

In [ ]:
print(classification_report(actual_values, list_pred, digits=4))

              precision    recall  f1-score   support

           0     0.5882    1.0000    0.7407        10
           1     1.0000    0.3000    0.4615        10

    accuracy                         0.6500        20
   macro avg     0.7941    0.6500    0.6011        20
weighted avg     0.7941    0.6500    0.6011        20

